https://edinburghcyclehire.com/
Souřadnnice sanic jsou uváděny na 14 respektive 16 desetinných míst (Decimal degrees in WGS84).



In [251]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 500)
pd.set_option("display.precision", 14)

In [252]:
import numpy as np
import pandas as pd

In [253]:
rides = pd.read_csv("201809-202010.csv", parse_dates=[0,1], usecols = ["started_at","ended_at","duration","start_station_id", "end_station_id"])    

In [254]:
# Vytvoření tabulky obsahující stanice = načtení z csv a dopočítání vzdálenosti mezi stanicemi.
# Zdrojová data obsahují nekonzistence v názvech a popisech stanic. Jejich zeměpisná šířka a délka také není u všech údajů stejná. To přičítám odchylce GPS a různému umístění kola.
# Některé stanice jsou jen v konečné stanici (například stanice id=242). 
# Celková tabulka stations je vytvořena spojením informací ze "start" a "end" slouců. Duplicity jsou vyřešeny vybráním prvního popisu a průměrem souřadnic.
# Načtení informací o "start" a "end" stanicích:
stations = pd.read_csv("201809-202010.csv", index_col = "start_station_id", usecols = ["start_station_id","start_station_name","start_station_description","start_station_latitude", "start_station_longitude"])   
end_stations = pd.read_csv("201809-202010.csv", index_col = "end_station_id", usecols = ["end_station_id","end_station_name","end_station_description","end_station_latitude", "end_station_longitude"])  
# Přejmenování sloupců a indexu:
column_names = {"start_station_name":"name","start_station_description":"description","start_station_latitude":"latitude", "start_station_longitude":"longitude"}
stations.rename(columns=column_names, index={"start_station_id": "idd"}, inplace=True)
stations.index.names = ['id']
end_column_names = {"end_station_name":"name","end_station_description":"description","end_station_latitude":"latitude", "end_station_longitude":"longitude"}
end_stations.rename(columns=end_column_names, index={'end_station_id': 'id'}, inplace=True)
end_stations.index.names = ['id']
# Sloučení informací o "start" a "end" stanicích a jejich setřídění:
stations = stations.append(end_stations)
stations.sort_index(axis="index", inplace = True)
# Výpočet průměrných koorinátů a přidání k prvním názvům a popisům stanic:
stations_coord_mean = stations.groupby(["id"]).mean()
stations = stations.groupby(["id"]).first()
stations["latitude"] = stations_coord_mean["latitude"]
stations["longitude"] = stations_coord_mean["longitude"]
# Vymazání dočasných df:
end_stations = end_stations[0:0]
stations_coord_mean = stations_coord_mean[0:0]

In [255]:
stations.head(5)

,name,description,latitude,longitude
id,,,,
171,George Square,"George Square, south side in front of University library",55.94308406300859,-3.18831107272196
183,Waverley Bridge,near the top of the South ramp,55.95158115432231,-3.19159561359431
189,City Chambers,City Chambers Quadrangle,55.95010933018049,-3.19025777009335
225,Waverley Court,On Waverley Court forecourt,55.95173449698063,-3.18417853540628
241,Depot,Depot,55.97237325804326,-3.15583315076697
